# Supply Chain On-Time Delivery Optimization

**Goal**: Improve on-time delivery performance through targeted interventions

This notebook demonstrates using Intervention Search to optimize supply chain operations by identifying which operational levers to adjust for maximum improvement in on-time delivery.

## 1. Load Supply Chain Data

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('data/supply_chain_data.csv')
print(f"Loaded {len(df)} supply chain orders")
print(f"\nKey Metrics:")
print(f"  • On-time delivery rate: {df['on_time_delivery'].mean()*100:.1f}%")
print(f"  • Avg delivery score: {df['on_time_delivery_score'].mean():.1f}")
print(f"  • Avg production efficiency: {df['production_efficiency'].mean():.1f}%")
print(f"  • Avg order fulfillment time: {df['order_fulfillment_time'].mean():.1f} days")
df.head()

## 2. Define Supply Chain Causal Graph

**Causal Structure:**
- `supplier_reliability → raw_material_quality → production_efficiency`
- `production_efficiency → on_time_delivery_score`
- `warehouse_capacity → inventory_turnover → order_fulfillment_time`
- `demand_variability → safety_stock → inventory_turnover`
- `lead_time → safety_stock`
- `transportation_mode → delivery_speed → on_time_delivery_score`
- `transportation_mode → shipping_cost`
- `order_fulfillment_time → on_time_delivery_score`

In [ ]:
# Define nodes
nodes = [
    'supplier_reliability', 'warehouse_capacity', 'transportation_mode',
    'demand_variability', 'lead_time', 'order_quantity',
    'raw_material_quality', 'production_efficiency', 'safety_stock',
    'inventory_turnover', 'delivery_speed', 'shipping_cost',
    'order_fulfillment_time', 'on_time_delivery_score'
]

# Define causal edges
edges = [
    ('supplier_reliability', 'raw_material_quality'),
    ('raw_material_quality', 'production_efficiency'),
    ('warehouse_capacity', 'inventory_turnover'),
    ('demand_variability', 'safety_stock'),
    ('lead_time', 'safety_stock'),
    ('safety_stock', 'inventory_turnover'),
    ('inventory_turnover', 'order_fulfillment_time'),
    ('production_efficiency', 'order_fulfillment_time'),
    ('transportation_mode', 'delivery_speed'),
    ('transportation_mode', 'shipping_cost'),
    ('order_quantity', 'shipping_cost'),
    ('production_efficiency', 'on_time_delivery_score'),
    ('order_fulfillment_time', 'on_time_delivery_score'),
    ('delivery_speed', 'on_time_delivery_score')
]

# Create adjacency matrix
adj_matrix = pd.DataFrame(0, index=nodes, columns=nodes)
for parent, child in edges:
    adj_matrix.loc[parent, child] = 1

print("Supply Chain Causal Graph:")
print(f"  • Nodes: {len(nodes)}")
print(f"  • Edges: {len(edges)}")
print(f"  • Target: on_time_delivery_score")

## 3. Train Causal Models

In [ ]:
from ht_categ import HT, HTConfig

# Train HT model
config = HTConfig(graph=adj_matrix, model_type='XGBoost')
ht_model = HT(config)
ht_model.train(df)

print("✓ Causal models trained successfully\n")

# Display model quality
print("Model Quality Assessment (R²):")
print("-" * 50)
quality_tiers = {'High (R²>0.7)': [], 'Medium (0.5-0.7)': [], 'Low (<0.5)': []}

for node, metrics in ht_model.model_metrics.items():
    if 'r2' in metrics:
        r2 = metrics['r2']
        if r2 > 0.7:
            quality_tiers['High (R²>0.7)'].append((node, r2))
        elif r2 > 0.5:
            quality_tiers['Medium (0.5-0.7)'].append((node, r2))
        else:
            quality_tiers['Low (<0.5)'].append((node, r2))

for tier, nodes_list in quality_tiers.items():
    if nodes_list:
        print(f"\n{tier}:")
        for node, r2 in sorted(nodes_list, key=lambda x: x[1], reverse=True):
            print(f"  • {node}: {r2:.3f}")

## 4. Find Interventions to Improve On-Time Delivery by 15%

In [ ]:
from intervention_search import InterventionSearch

# Initialize searcher
searcher = InterventionSearch(
    graph=ht_model.graph,
    ht_model=ht_model,
    n_simulations=1000
)

# Search for interventions
results = searcher.find_interventions(
    target_outcome='on_time_delivery_score',
    target_change=15.0,  # +15% improvement
    tolerance=3.0,
    confidence_level=0.90,
    max_intervention_pct=25.0,
    verbose=True
)

## 5. Analyze Best Intervention

In [ ]:
best = results['best_intervention']

print("\n" + "="*70)
print("OPTIMAL INTERVENTION STRATEGY")
print("="*70)
print(f"\n🎯 Target: Improve on-time delivery score by 15%")
print(f"\n📊 Recommended Intervention: {', '.join(best['nodes'])}")

print(f"\n🔧 Required Changes:")
for node, change in best['required_pct_changes'].items():
    baseline = ht_model.baseline_stats[node]['mean']
    new_value = baseline * (1 + change/100)
    direction = '↑' if change > 0 else '↓'
    print(f"  {direction} {node}: {change:+.1f}%")
    print(f"     Current: {baseline:.2f} → Target: {new_value:.2f}")

print(f"\n📈 Expected Impact:")
print(f"  • Predicted improvement: {best['actual_effect']:+.1f}%")
print(f"  • 90% Confidence Interval: [{best['ci_90'][0]:+.1f}%, {best['ci_90'][1]:+.1f}%]")
print(f"  • 50% Confidence Interval: [{best['ci_50'][0]:+.1f}%, {best['ci_50'][1]:+.1f}%]")
print(f"  • Overall Confidence: {best['confidence']:.0%}")
print(f"  • Model Quality Grade: {best['quality']['overall_grade']}")

print(f"\n✅ Status: {'APPROVED' if best['within_tolerance'] else 'NEEDS REVIEW'}")
print("="*70)

## 6. Compare Top 5 Intervention Options

In [ ]:
print("\n📋 Top 5 Intervention Options:\n")
print("-" * 80)

comparison_data = []
for i, candidate in enumerate(results['all_candidates'][:5], 1):
    print(f"\n{i}. {', '.join(candidate['nodes'])}")
    print(f"   Effect: {candidate['actual_effect']:+.1f}% | "
          f"Uncertainty: ±{candidate['uncertainty']:.1f}% | "
          f"Confidence: {candidate['confidence']:.0%}")
    print(f"   Quality: {candidate['quality']['overall_grade']} | "
          f"Changes: {candidate['required_pct_changes']}")
    
    comparison_data.append({
        'Rank': i,
        'Intervention': ', '.join(candidate['nodes']),
        'Effect': f"{candidate['actual_effect']:+.1f}%",
        'Confidence': f"{candidate['confidence']:.0%}",
        'Grade': candidate['quality']['overall_grade']
    })

# Create comparison DataFrame
comp_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)
print(comp_df.to_string(index=False))

## 7. Operational Feasibility Analysis

Evaluate practical implementation challenges for top interventions

In [ ]:
# Define implementation difficulty scores (1=easy, 5=very difficult)
implementation_difficulty = {
    'supplier_reliability': 4,  # Requires supplier changes/switching
    'warehouse_capacity': 5,    # Major infrastructure investment
    'transportation_mode': 3,   # Operational change, some cost
    'demand_variability': 5,    # Hard to control (external)
    'lead_time': 4,             # Process re-engineering
    'order_quantity': 2,        # Policy change
    'production_efficiency': 3,  # Process improvement
    'inventory_turnover': 3,    # Inventory management changes
}

print("\nImplementation Feasibility Analysis:\n")
print("-" * 70)

for i, candidate in enumerate(results['all_candidates'][:5], 1):
    nodes = candidate['nodes']
    avg_difficulty = np.mean([implementation_difficulty.get(n, 3) for n in nodes])
    
    feasibility = "Easy" if avg_difficulty < 2.5 else "Moderate" if avg_difficulty < 3.5 else "Challenging"
    
    print(f"{i}. {', '.join(nodes)}")
    print(f"   Implementation: {feasibility} (score: {avg_difficulty:.1f}/5)")
    print(f"   Expected Effect: {candidate['actual_effect']:+.1f}%")
    print(f"   Confidence: {candidate['confidence']:.0%}")
    print()

## 8. Causal Path Analysis

Understanding how interventions propagate through the supply chain

In [ ]:
# Analyze causal paths for best intervention
if 'path_analysis' in results and results['path_analysis']:
    path_info = results['path_analysis']
    
    print("\nCausal Path Analysis:")
    print("="*70)
    print(f"Total paths evaluated: {path_info.get('total_paths', 'N/A')}")
    print(f"High quality paths (R²>0.7): {path_info.get('high_quality_paths', 'N/A')}")
    print(f"Average path quality: {path_info.get('avg_path_quality', 0):.3f}")
    print(f"\nThis indicates the reliability of causal effect propagation")
    print(f"from intervention nodes to the target outcome.")
else:
    print("\nPath analysis not available in results.")

## 9. Sensitivity Analysis: Different Target Levels

In [ ]:
# Compare interventions for different improvement targets
print("\nSensitivity Analysis: Testing different improvement targets\n")
print("-" * 70)

targets = [10.0, 15.0, 20.0]
sensitivity_results = []

for target in targets:
    result = searcher.find_interventions(
        target_outcome='on_time_delivery_score',
        target_change=target,
        tolerance=3.0,
        confidence_level=0.90,
        max_intervention_pct=30.0,
        verbose=False
    )
    
    if result['best_intervention']:
        best = result['best_intervention']
        sensitivity_results.append({
            'Target': f"+{target}%",
            'Best Node': ', '.join(best['nodes']),
            'Required Change': f"{list(best['required_pct_changes'].values())[0]:+.1f}%",
            'Confidence': f"{best['confidence']:.0%}",
            'Grade': best['quality']['overall_grade']
        })

if sensitivity_results:
    sens_df = pd.DataFrame(sensitivity_results)
    print(sens_df.to_string(index=False))
    
    print("\n💡 Insight: Larger targets may require interventions on different nodes")
    print("   or larger magnitude changes with potentially lower confidence.")

## 10. Key Insights & Action Plan

### 🎯 Key Findings:

1. **Primary Leverage Points**: The analysis identified which operational variables have the strongest causal impact on on-time delivery
2. **Model Reliability**: High-quality models (R² > 0.7) provide more reliable intervention predictions
3. **Trade-offs**: Some interventions are more effective but harder to implement
4. **Uncertainty Quantification**: Confidence intervals help assess risk of intervention strategies

### 📋 Recommended Action Plan:

**Phase 1 - Quick Wins (0-3 months):**
- Implement interventions with "Easy" or "Moderate" feasibility ratings
- Focus on operational changes that don't require major infrastructure investment
- Monitor actual vs. predicted improvements

**Phase 2 - Strategic Improvements (3-12 months):**
- Plan and execute more challenging interventions (supplier relationships, process re-engineering)
- Use learnings from Phase 1 to calibrate predictions
- Consider multi-node interventions for robust improvements

**Phase 3 - Continuous Optimization (Ongoing):**
- Retrain models with new data periodically
- Re-run intervention search as operational conditions change
- Track long-term trends and adjust strategies

### ⚠️ Important Considerations:

- **External Factors**: Some variables (demand variability) are partially outside direct control
- **Cost-Benefit**: Balance intervention difficulty with expected impact
- **Model Quality**: Prioritize interventions along high-quality causal paths
- **Validation**: Test predictions with pilot programs before full deployment

## Summary

This notebook demonstrated:
- ✅ Loading and analyzing supply chain operational data
- ✅ Building comprehensive causal graph for supply chain processes
- ✅ Training causal models with quality assessment
- ✅ Finding optimal interventions with uncertainty quantification
- ✅ Evaluating implementation feasibility
- ✅ Conducting sensitivity analysis across different targets
- ✅ Developing phased action plans based on insights

**Key Advantages of Intervention Search:**
1. **Proper Uncertainty Quantification**: Monte Carlo simulation provides realistic confidence intervals
2. **Model Quality Gating**: Filters unreliable interventions based on model R²
3. **Causal Path Analysis**: Identifies which paths are reliable vs. unreliable
4. **Multi-Objective Ranking**: Balances accuracy, uncertainty, quality, and simplicity
5. **Out-of-Distribution Detection**: Flags interventions that push variables outside training data